# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [3]:
# Your code here
pd.read_sql("""
SELECT firstName, lastName
 FROM employees
      JOIN offices
      ON employees.officeCode = offices.officeCode
      LIMIT 10;
""", conn)

,firstName,lastName
0,Diane,Murphy
1,Mary,Patterson
2,Jeff,Firrelli
3,William,Patterson
4,Gerard,Bondur
5,Anthony,Bow
6,Leslie,Jennings
7,Leslie,Thompson
8,Julie,Firrelli
9,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [4]:
# Your code here
pd.read_sql("""
SELECT 
    o.officeCode,
    o.city,
    COUNT(e.employeeNumber) AS numberOfEmployees
FROM 
    offices o
LEFT JOIN 
    employees e ON o.officeCode = e.officeCode
GROUP BY 
    o.officeCode, o.city
HAVING 
    numberOfEmployees = 0;
""", conn)

,officeCode,city,numberOfEmployees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [5]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

'\nHow many customers are there per office?\n'

In [14]:
"""
Which product line generates the highest average profit per item sold?
"""

# Your code here
pd.read_sql("""
SELECT 
    p.productLine,
    AVG(od.priceEach - p.buyPrice) AS avg_profit_per_item,
    SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)) AS total_profit
FROM 
    products p
JOIN 
    orderdetails od ON p.productCode = od.productCode
GROUP BY 
    p.productLine
ORDER BY 
    avg_profit_per_item DESC;
""", conn)

,productLine,avg_profit_per_item,total_profit
0,Classic Cars,42.732574,1526212.20
1,Motorcycles,36.473370,469255.30
2,Trucks and Buses,36.380162,400553.22
3,Vintage Cars,32.338402,737268.33
4,Ships,30.754776,261289.47
5,Planes,30.698036,365960.71
6,Trains,23.217160,65341.02


In [15]:
"""
Which month typically has the highest sales volume across all years?
"""

# Your code here
pd.read_sql("""
SELECT 
    strftime('%m', o.orderDate) AS month,
    SUM(od.quantityOrdered) AS total_units_sold
FROM 
    orders o
JOIN 
    orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY 
    month
ORDER BY 
    total_units_sold DESC;
""", conn)

,month,total_units_sold
0,11,21540
1,10,11214
2,05,9464
3,03,8294
4,01,7997
5,02,7959
6,04,7906
7,12,7769
8,08,6538
9,07,5721


In [16]:
"""
What percentage of customers are served by each sales representative?
"""

# Your code here
pd.read_sql("""
SELECT 
    CONCAT(e.firstName, ' ', e.lastName) AS sales_rep,
    COUNT(c.customerNumber) AS customer_count,
    ROUND(COUNT(c.customerNumber) * 100.0 / (SELECT COUNT(*) FROM customers), 2) AS percentage
FROM 
    employees e
LEFT JOIN 
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
WHERE 
    e.jobTitle = 'Sales Rep'
GROUP BY 
    e.employeeNumber
ORDER BY 
    customer_count DESC;
""", conn)

,sales_rep,customer_count,percentage
0,Pamela Castillo,10,8.20
1,Barry Jones,9,7.38
2,Larry Bott,8,6.56
3,George Vanauf,8,6.56
4,Gerard Hernandez,7,5.74
5,Foon Yue Tseng,7,5.74
6,Martin Gerard,6,4.92
7,Loui Bondur,6,4.92
8,Steve Patterson,6,4.92
9,Julie Firrelli,6,4.92


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [6]:
# Your code here
pd.read_sql("""
SELECT DISTINCT
    e.employeeNumber,
    CONCAT(e.firstName, ' ', e.lastName) AS employeeName,
    p.productName
FROM 
    employees e
JOIN 
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN 
    orders o ON c.customerNumber = o.customerNumber
JOIN 
    orderdetails od ON o.orderNumber = od.orderNumber
JOIN 
    products p ON od.productCode = p.productCode
ORDER BY 
    employeeName, p.productName;
""", conn)

,employeeNumber,employeeName,productName
0,1611,Andy Fixter,18th Century Vintage Horse Carriage
1,1611,Andy Fixter,1900s Vintage Bi-Plane
2,1611,Andy Fixter,1900s Vintage Tri-Plane
3,1611,Andy Fixter,1911 Ford Town Car
4,1611,Andy Fixter,1913 Ford Model T Speedster
...,...,...,...
1363,1216,Steve Patterson,The Mayflower
1364,1216,Steve Patterson,The Queen Mary
1365,1216,Steve Patterson,The Schooner Bluenose
1366,1216,Steve Patterson,The Titanic


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [7]:
# Your code here
pd.read_sql("""
SELECT 
    e.employeeNumber,
    e.lastName,
    e.firstName,
    SUM(od.quantityOrdered) AS totalProductsSold
FROM 
    employees e
JOIN 
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN 
    orders o ON c.customerNumber = o.customerNumber
JOIN 
    orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY 
    e.employeeNumber, e.lastName, e.firstName
ORDER BY 
    e.lastName ASC;
""", conn)

,employeeNumber,lastName,firstName,totalProductsSold
0,1337,Bondur,Loui,6186
1,1501,Bott,Larry,8205
2,1401,Castillo,Pamela,9290
3,1188,Firrelli,Julie,4227
4,1611,Fixter,Andy,6246
5,1702,Gerard,Martin,4180
6,1370,Hernandez,Gerard,14231
7,1165,Jennings,Leslie,11854
8,1504,Jones,Barry,7486
9,1612,Marsh,Peter,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [11]:
# Your code here
pd.read_sql("""
SELECT 
    e.employeeNumber,
    e.lastName,
    e.firstName,
    COUNT(DISTINCT od.productCode) AS uniqueProductsSold
FROM 
    employees e
JOIN 
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN 
    orders o ON c.customerNumber = o.customerNumber
JOIN 
    orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY 
    e.employeeNumber, e.lastName, e.firstName
HAVING 
    uniqueProductsSold >= 200
ORDER BY 
    uniqueProductsSold DESC, e.lastName ASC;
""", conn)

,employeeNumber,lastName,firstName,uniqueProductsSold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!